<a href="https://colab.research.google.com/github/JonJonesBR/RPG_GEMINI_TTS_STREAMLIT/blob/main/RPG_GEMINI_TTS_STREAMLIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q streamlit google-generativeai edge-tts pillow nest_asyncio pyngrok

In [ ]:
!rm -f /content/rpg_streamlit_app.py
print("Arquivo /content/rpg_streamlit_app.py DELETADO (se existia).")
!ls /content/

In [ ]:
%%writefile rpg_streamlit_app.py

# RPG_STREAMLIT_APP_V1_6
import streamlit as st
import google.generativeai as genai
import time
import os
import io
import asyncio
import re
import sys
import random
import urllib.parse # Para codificar URLs
import requests # Para chamadas HTTP (geração de imagem por IA)
import base64   # Para decodificar imagem base64

try:
    import nest_asyncio
    nest_asyncio.apply()
    print("nest_asyncio aplicado para Streamlit.")
except ImportError:
    print("AVISO: nest_asyncio não encontrado.")
except RuntimeError as e:
    if "cannot apply nest_asyncio" in str(e) or "loop is already running" in str(e):
        print(f"AVISO nest_asyncio: {e}")
    else:
        raise

_edge_tts_available = False
try:
    import edge_tts
    _edge_tts_available = True
except ImportError:
    print("AVISO: Biblioteca edge-tts não encontrada. Funcionalidade de voz limitada.")

# CONFIGURAÇÕES GLOBAIS
API_KEY_FILENAME = ".gemini_api_key_streamlit.txt"
DURATION_OPTIONS_CONFIG = {
    "1": {"name": "Curta (~10 turnos)", "id": "curta", "turns": 10, "extension": 5},
    "2": {"name": "Média (~20 turnos)", "id": "media", "turns": 20, "extension": 10},
    "3": {"name": "Longa (~40 turnos)", "id": "longa", "turns": 40, "extension": 15}
}
RPG_THEMES_CONFIG = {
    "medieval": "Medieval Clássico", "fantasia": "Alta Fantasia", "futurista": "Futurista",
    "cyberpunk": "Cyberpunk Noir", "steampunk": "Steampunk", "velho_oeste": "Velho Oeste",
    "pos_apoc": "Pós-Apocalíptico", "investigacao": "Investigação Sobrenatural",
    "personalizado": " Tema Personalizado +"
}
THEME_AVATARS = {
    "medieval": "⚔️", "fantasia": "🦄", "futurista": "🚀", "cyberpunk": "🤖",
    "steampunk": "⚙️", "velho_oeste": "🤠", "pos_apoc": "☢️", "investigacao": "🕵️",
    "personalizado": "✨", "default": "🐉"
}
GM_PERSONALITIES = {
    "neutro": {"name": "Neutro e Direto", "desc": "Um mestre objetivo, focado na progressão da história."},
    "epico": {"name": "Épico e Descritivo", "desc": "Narração grandiosa, rica em detalhes e emoção."},
    "sombrio": {"name": "Sério e Sombrio", "desc": "Tom pesado, suspense, perigos constantes."},
    "engracado": {"name": "Leve e Sagaz", "desc": "Inclui humor, situações inusitadas e diálogos divertidos."},
    "misterioso": {"name": "Enigmático e Misterioso", "desc": "Um mestre que fala por enigmas e deixa pistas sutis."}
}
NARRATIVE_FOCUSES = {
    "forca": "Força Bruta", "agilidade": "Agilidade Felina", "intelecto": "Mente Afiada",
    "carisma": "Língua Habilidosa", "percepcao": "Percepção Aguçada", "resistencia": "Vigor Inabalável"
}
PLAYER_GENDERS = {
    "masculino": "Masculino", "feminino": "Feminino", "neutro": "Neutro / Não especificado"
}
AVAILABLE_GEMINI_MODELS_TEXT = {
    "gemini-1.5-flash-latest": "Gemini 1.5 Flash (Rápido e Eficiente)",
    "gemini-2.0-flash-lite": "Gemini 2.0 Flash Lite (Robusto)"
}
DEFAULT_GEMINI_MODEL_TEXT_KEY = "gemini-2.0-flash-lite"
DEFAULT_IMAGE_GENERATION_MODEL = "gemini-2.0-flash-preview-image-generation" # User specified model

SFX_PATHS = {
    "new_item": "/content/sfx_new_item.mp3",
    "objective_updated": "/content/sfx_objective_updated.mp3"
}

THEME_CLASSES = {
    "medieval": ["Cavaleiro(a)", "Mago(a)", "Arqueiro(a)", "Clérigo(a)", "Bardo(a)"],
    "fantasia": ["Guerreiro(a) Élfico(a)", "Feiticeira Elemental", "Ladino(a) Halfling", "Druida Metamorfo(a)", "Paladino(a) da Luz"],
    "futurista": ["Piloto(a) de Mecha", "Engenheiro(a) Cibernético(a)", "Agente Secreto(a) Intergaláctico(a)", "Explorador(a) Xenobiólogo(a)", "Diplomata Estelar"],
    "cyberpunk": ["Samurai das Ruas", "Netrunner (Hacker)", "Detetive Bio-modificado(a)", "Contrabandista de Dados", "Médico(a) de Rua"],
    "steampunk": ["Inventor(a) Genial", "Caçador(a) de Autômatos", "Aeronauta Destemido(a)", "Detetive Mecânico(a)", "Relojoeiro(a) Arcano(a)"],
    "velho_oeste": ["Pistoleiro(a)", "Caçador(a) de Recompensas", "Médico(a) Itinerante", "Jogador(a) Astuto(a)", "Xerife Obstinado(a)"],
    "pos_apoc": ["Sobrevivente Nato(a)", "Batedor(a) de Ermos", "Mecânico(a) de Sucata", "Saqueador(a) Astuto(a)", "Líder Comunitário(a)"],
    "investigacao": ["Detetive Particular", "Estudioso(a) do Oculto", "Jornalista Investigativo(a)", "Psíquico(a) Sensitivo(a)", "Exorcista Cético(a)"]
}

generation_config_text_default = {"temperature": 0.8, "top_p": 0.9, "top_k": 35}
safety_settings_text_default = [{"category": c, "threshold": "BLOCK_MEDIUM_AND_ABOVE"} for c in ["HARM_CATEGORY_HARASSMENT", "HARM_CATEGORY_HATE_SPEECH", "HARM_CATEGORY_SEXUALLY_EXPLICIT", "HARM_CATEGORY_DANGEROUS_CONTENT"]]

async def generate_audio_bytes(text, voice_name="pt-BR-ThalitaNeural"):
    if not _edge_tts_available or not text or not text.strip(): return None
    text_cleaned = re.sub(r'[\*#_]', '', text)
    text_cleaned = re.sub(r'<i>\(.*?\)</i>', '', text_cleaned)
    text_cleaned = re.sub(r'<span.*?</span>', '', text_cleaned)
    text_cleaned = re.sub(r'\s+', ' ', text_cleaned).strip()
    if not text_cleaned: return None

    try:
        communicate = edge_tts.Communicate(text_cleaned, voice_name)
        audio_bytes_io = io.BytesIO()
        async for chunk in communicate.stream():
            if chunk["type"] == "audio": audio_bytes_io.write(chunk["data"])
        audio_bytes_io.seek(0)
        return audio_bytes_io.getvalue() if audio_bytes_io.getbuffer().nbytes > 0 else None
    except Exception as e_tts:
        print(f"Erro no TTS Edge: {e_tts}")
        if 'streamlit' in sys.modules: st.toast(f"⚠️ Erro na geração de voz: {e_tts}", icon="🔊")
        return None

def call_gemini_api(prompt_text, model_instance, generation_config=None, safety_settings=None):
    if not model_instance:
        if 'streamlit' in sys.modules: st.error("Modelo Gemini não configurado.")
        return "Erro: Modelo Gemini não configurado."

    gen_config_to_use = generation_config if generation_config else generation_config_text_default
    safety_to_use = safety_settings if safety_settings else safety_settings_text_default

    try:
        response = model_instance.generate_content(
            prompt_text,
            generation_config=gen_config_to_use,
            safety_settings=safety_to_use
        )
        return response.text
    except Exception as e:
        if 'streamlit' in sys.modules: st.error(f"Erro na API Gemini: {e}")
        return f"Erro na API Gemini: {e}"

async def async_get_image_search_terms_and_google_url(narrative_text, text_model_instance):
    """Gera termos de busca e um URL do Google Imagens."""
    if not narrative_text or not text_model_instance:
        return None, "Sem narrativa para gerar descrição de imagem."

    image_desc_prompt = f"""
Baseado na seguinte narração de uma cena de RPG, crie uma descrição MUITO CURTA e OTIMIZADA (5-7 palavras-chave)
para encontrar uma imagem representativa no Google Imagens. Foque nos elementos visuais chave.
Evite nomes próprios, a menos que seja uma figura pública ou personagem muito conhecido.
Exemplos:
"floresta escura cabana luz distante noite"
"guerreiro armadura dragão vermelho caverna batalha"
"mercado medieval barracas pessoas fantasia"

Narração:
"{narrative_text[:500]}"

Termos de Busca para Imagem (5-7 palavras-chave):
"""
    try:
        image_search_terms_raw = call_gemini_api(image_desc_prompt, text_model_instance)
        if "Erro:" in image_search_terms_raw or not image_search_terms_raw.strip():
            print("Falha ao gerar termos de busca para imagem com o modelo de texto.")
            return None, "Falha ao criar termos de busca."

        image_search_terms = image_search_terms_raw.strip().replace("\n", " ")
        print(f"DEBUG: Termos de busca para imagem gerados: {image_search_terms}")

        encoded_search_terms_for_google = urllib.parse.quote_plus(image_search_terms)
        google_images_url = f"https://www.google.com/search?tbm=isch&q={encoded_search_terms_for_google}"

        return google_images_url, image_search_terms
    except Exception as e:
        print(f"Erro em async_get_image_search_terms_and_google_url: {e}")
        return None, "Erro ao gerar termos de busca."

async def generate_ai_image_async(prompt_text, api_key, model_name=DEFAULT_IMAGE_GENERATION_MODEL):
    """Gera uma imagem usando o modelo especificado (padrão: gemini-2.0-flash-preview-image-generation) e retorna os bytes da imagem."""
    print(f"DEBUG: Tentando gerar imagem com IA. Modelo: {model_name}, Prompt: '{prompt_text}'")
    if not api_key:
        st.warning("Chave API não configurada. Não é possível gerar imagem com IA.")
        print("DEBUG: Chave API ausente para geração de imagem com IA.")
        return None
    if not prompt_text or not prompt_text.strip():
        st.warning("Texto de prompt vazio. Não é possível gerar imagem com IA.")
        print("DEBUG: Prompt vazio para geração de imagem com IA.")
        return None

    # Usando o endpoint :predict conforme a estrutura para Imagen, mas com o modelo Gemini especificado
    api_url = f"https://generativelanguage.googleapis.com/v1beta/models/{model_name}:predict?key={api_key}"

    payload = {
        "instances": [{"prompt": prompt_text}],
        "parameters": {"sampleCount": 1}
    }
    headers = {'Content-Type': 'application/json'}

    try:
        response = await asyncio.to_thread(requests.post, api_url, json=payload, headers=headers, timeout=120)
        response.raise_for_status()
        result = response.json()
        print(f"DEBUG: Resposta da API de Geração de Imagem ({model_name}): {result}")

        if result.get("predictions") and len(result["predictions"]) > 0 and result["predictions"][0].get("bytesBase64Encoded"):
            base64_image_data = result["predictions"][0]["bytesBase64Encoded"]
            image_bytes = base64.b64decode(base64_image_data)
            print(f"DEBUG: Imagem gerada com IA com sucesso (bytes obtidos).")
            return image_bytes
        else:
            error_detail = result.get("error", {}).get("message", f"Estrutura de resposta da API {model_name} inesperada.")
            print(f"DEBUG: Falha na API de Imagem {model_name}: {error_detail}. Resposta: {result}")
            st.toast(f"⚠️ Falha na API de Imagem ({model_name}): {error_detail}", icon="🖼️")
            return None
    except requests.exceptions.Timeout:
        error_message = f"Timeout ao chamar a API de Imagem ({model_name})."
        print(f"DEBUG: {error_message}")
        st.toast(f"⏳ {error_message}", icon="🖼️")
        return None
    except requests.exceptions.RequestException as e:
        error_message = f"Erro na chamada da API de Imagem ({model_name}): {e}"
        if e.response is not None:
            try:
                error_detail = e.response.json().get("error", {}).get("message", e.response.text)
                error_message += f" - Detalhe: {error_detail}"
            except ValueError:
                error_message += f" - Resposta: {e.response.text}"
        print(f"DEBUG: {error_message}")
        st.toast(f"🔥 {error_message}", icon="🖼️")
        return None
    except Exception as e:
        error_message = f"Erro inesperado ao gerar imagem com IA ({model_name}): {e}"
        print(f"DEBUG: {error_message}")
        st.toast(f"🔥 {error_message}", icon="🖼️")
        return None

def parse_gemini_setup_response(response_text):
    data = {
        "world_lore": "Mundo padrão.", "character_lore": "Herói padrão.",
        "current_location": "Local padrão.", "current_objective": "Objetivo padrão.",
        "player_inventory": ["Item padrão"], "known_npcs": {}
    }
    if not response_text or not response_text.strip() or "Erro:" in response_text: return data

    lines = response_text.strip().split('\n')
    for line in lines:
        if ":" not in line: continue
        prefix, value = line.split(":", 1); value = value.strip(); prefix_upper = prefix.strip().upper()

        if prefix_upper == "LORE_MUNDO": data["world_lore"] = value
        elif prefix_upper == "LORE_PERSONAGEM": data["character_lore"] = value
        elif prefix_upper in ["LOCALIZAÇÃO", "LOCALIZACAO"]: data["current_location"] = value
        elif prefix_upper == "OBJETIVO": data["current_objective"] = value
        elif prefix_upper in ["INVENTÁRIO", "INVENTARIO"]:
            items_str = value.replace('[','').replace(']',''); parsed_items = [item.strip() for item in items_str.split(',') if item.strip()]
            if parsed_items: data["player_inventory"] = parsed_items
    return data

def generate_status_markdown(gs):
    if not gs or not gs.get("player_name"): return "Aguardando início da aventura..."

    focus_texts = [NARRATIVE_FOCUSES[key] for key in gs.get('narrative_focus_keys', [])]
    focus_display = ', '.join(focus_texts) if focus_texts else "Nenhum"
    gender_display = PLAYER_GENDERS.get(gs.get("player_gender_key", "neutro"), "Não especificado")

    col1_lines = [
        f"**Aventureiro(a):** {gs.get('player_name', 'N/A')}",
        f"**Gênero:** {gender_display}",
        f"**Classe:** {gs.get('player_class', 'N/A')}",
        f"**Focos:** {focus_display}"
    ]
    col2_lines = [
        f"**Tema:** {gs.get('rpg_theme_config_name', 'N/A')}",
        f"**Mestre:** {gs.get('gm_personality_name', 'Neutro')}",
        f"**Turno:** {gs.get('current_turn', 0)} / {gs.get('max_turns', 'N/A')} ({gs.get('chosen_duration_name', 'N/A')})"
    ]
    if gs.get("adventure_extended"):
        col2_lines.append(f"<span style='color: lime; font-size: small;'>(Aventura Estendida!)</span>")

    col1_html = "<br>".join(col1_lines); col2_html = "<br>".join(col2_lines)
    status_html = f"""
<div style="background-color: #222227; padding: 15px; border-radius: 10px; border: 1px solid #444;">
    <h4 style="color: #1E90FF; margin-bottom: 10px; text-align: center;">FICHA DO PERSONAGEM</h4>
    <table style="width:100%;"><tr>
        <td style="width:50%; vertical-align:top; padding-right:10px;">{col1_html}</td>
        <td style="width:50%; vertical-align:top; padding-left:10px;">{col2_html}</td>
    </tr></table>
    <hr style="border-color: #444; margin-top: 10px; margin-bottom: 10px;">
    <div style="text-align: left;">
        <strong>Objetivo Principal:</strong> {gs.get('current_objective', 'N/D')}<br>
        <strong>Localização Atual:</strong> {gs.get('current_location', 'N/D')}
    </div>
</div>"""
    return status_html

def init_session_state():
    defaults = {
        "api_key": None, "api_configured": False, "gemini_model_instance": None,
        "gemini_model_text_key": DEFAULT_GEMINI_MODEL_TEXT_KEY,
        "app_stage": "api_config", "gs": {}, "chat_history": [], "internal_ai_history": [],
        "last_audio_bytes": None, "bot_avatar": THEME_AVATARS["default"],
        "action_from_button": None, "sfx_to_play": None,
        "show_extend_adventure_options": False, "adventure_extended": False,
        "last_user_action_for_reroll": None
    }
    for key, value in defaults.items():
        if key not in st.session_state: st.session_state[key] = value

    gs_defaults = {
        "known_npcs": {}, "enable_unexpected_events": True,
        "current_action_options": [], "action_buttons_shown": False,
        "enable_image_search": False,
        "current_image_search_url": None,
        "current_image_search_terms": None,
        "current_generated_image_bytes": None,
        "image_processing_in_progress": False,
    }
    if not isinstance(st.session_state.gs, dict): st.session_state.gs = {}

    for gs_key, gs_default_value in gs_defaults.items():
        if gs_key not in st.session_state.gs:
            st.session_state.gs[gs_key] = gs_default_value

init_session_state()

st.set_page_config(page_title="RPG Gemini Adventure", layout="wide", initial_sidebar_state="expanded")
st.title("⚔️ RPG Textual v1.6 com IA Gemini & Voz 🐉") # VISUAL VERSION CHANGE

if st.session_state.sfx_to_play:
    sfx_file = st.session_state.sfx_to_play
    if os.path.exists(sfx_file):
        try:
            with open(sfx_file, "rb") as f: sfx_bytes = f.read()
            st.audio(sfx_bytes, format="audio/mp3", autoplay=True)
        except Exception as e: print(f"Erro ao tocar SFX {sfx_file}: {e}")
    else: print(f"Arquivo SFX não encontrado: {sfx_file}")
    st.session_state.sfx_to_play = None

async def process_player_action_async(user_action_param, gs_data_param, is_reroll=False):
    if not is_reroll:
        st.session_state.chat_history.append({"role": "user", "content": user_action_param})
        st.session_state.internal_ai_history.append({'role': 'user', 'parts': [user_action_param]})
        st.session_state.last_user_action_for_reroll = user_action_param
    else:
        if st.session_state.chat_history and st.session_state.chat_history[-1]["role"] == "assistant":
            st.session_state.chat_history.pop()
        if st.session_state.internal_ai_history and st.session_state.internal_ai_history[-1]["role"] == "model":
            st.session_state.internal_ai_history.pop()

    st.session_state.gs["current_image_search_url"] = None
    st.session_state.gs["current_image_search_terms"] = None
    st.session_state.gs["current_generated_image_bytes"] = None

    st.session_state.gs["current_action_options"] = []
    st.session_state.gs["action_buttons_shown"] = False

    spinner_message = "Mestre reflete sobre seus atos..."
    if "uso o item" in user_action_param.lower(): spinner_message = "Mestre observa o uso do item..."
    elif "analiso o item" in user_action_param.lower(): spinner_message = "Mestre detalha o item..."
    if is_reroll: spinner_message = "Mestre reconsidera os ventos do destino..."

    with st.spinner(spinner_message):
        hist_f_p = []
        temp_internal_history = list(st.session_state.internal_ai_history)
        system_entry = next((item for item in temp_internal_history if item.get('role') == 'system'), None)
        if system_entry: hist_f_p.append(f"SISTEMA: {system_entry['parts'][0]}")

        user_model_history = [item for item in temp_internal_history if item.get('role') in ['user', 'model']]
        for entry in user_model_history[-8:]:
            role_label = "JOGADOR" if entry['role'] == 'user' else "MESTRE"
            content = entry.get('parts', [''])[0] if entry.get('parts') else ""
            hist_f_p.append(f"{role_label}: {content}")
        hist_f = "\n\n".join(hist_f_p)

        ct, mt = gs_data_param.get('current_turn', 1), gs_data_param.get('max_turns', 10)
        cdn = gs_data_param.get('chosen_duration_name', 'N/A')
        rpg_tn = gs_data_param.get('rpg_theme_config_name', 'N/A')
        gm_pers_name = gs_data_param.get('gm_personality_name', 'Neutro')
        gm_pers_desc = gs_data_param.get('gm_personality_desc', '')
        player_gender_str = PLAYER_GENDERS.get(gs_data_param.get('player_gender_key', 'neutro'), "Neutro")
        narrative_focuses_keys = gs_data_param.get('narrative_focus_keys', [])
        narrative_focuses_str = ", ".join([NARRATIVE_FOCUSES[f] for f in narrative_focuses_keys if f in NARRATIVE_FOCUSES]) or "Nenhum"

        SITUACAO_ATUAL = gs_data_param.get('current_situation', 'Não definida')
        INVENTARIO = gs_data_param.get('player_inventory', [])
        OBJETIVO_ATUAL = gs_data_param.get('current_objective', 'Não definido')
        KNOWN_NPCS = gs_data_param.get('known_npcs', {})
        ENABLE_UNEXPECTED_EVENTS = gs_data_param.get("enable_unexpected_events", True)

        turn_guidance = ""
        is_antepenultimate_turn = (ct == mt - 2)
        if ct >= mt:
            turn_guidance = f"Este é o ÚLTIMO TURNO ({ct}/{mt}). Sua narração deve ser CONCLUSIVA e ÉPICA, finalizando a aventura atual."
        elif ct == mt - 1:
            turn_guidance = f"Este é o PENÚLTIMO TURNO ({ct}/{mt}). Prepare o clímax da aventura, encaminhando para uma conclusão."
        elif is_antepenultimate_turn and not gs_data_param.get("adventure_extended", False) and mt > 2 :
             turn_guidance = f"A aventura está se aproximando do fim (Turno {ct}/{mt}). Concentre-se nos eventos cruciais. O jogador poderá optar por estender a aventura."
        else:
            turn_guidance = f"Mantenha a narração com 1-3 parágrafos. Guie o jogador para o objetivo ou desenvolva a trama. (Turno {ct}/{mt})"

        action_specific_instruction = ""
        if "analiso o item" in user_action_param.lower():
            action_specific_instruction = "O jogador inspeciona um item. Descreva-o detalhadamente, incluindo sua aparência, história (se houver) e possíveis usos."
        elif "uso o item" in user_action_param.lower():
            action_specific_instruction = f"O jogador tenta usar o item: '{user_action_param.split(':')[-1].strip()}'. Descreva o resultado dessa tentativa. Se o item for consumido, remova-o do inventário."

        reroll_instruction = "O jogador pediu para você refazer sua última narração. Por favor, gere uma variação SIGNIFICATIVA da narração anterior, explorando um resultado ou tom diferente para a mesma ação do jogador." if is_reroll else ""

        unexpected_event_instruction = ""
        if ENABLE_UNEXPECTED_EVENTS and not is_reroll and not (user_action_param.startswith("Eu analiso o item") or user_action_param.startswith("Eu uso o item")):
            if random.random() < 0.20:
                unexpected_event_instruction = "OPCIONALMENTE, introduza um PEQUENO evento inesperado ou uma reviravolta sutil para adicionar mais dinamismo à cena (não precisa ser algo grandioso)."

        prompt_m_r = f"""Você é o Mestre de um RPG textual.
Estilo do Mestre: {gm_pers_name} ({gm_pers_desc}). Mantenha este estilo consistentemente.
{reroll_instruction}
{turn_guidance}
{action_specific_instruction}
{unexpected_event_instruction}

CONTEXTO ATUAL DA AVENTURA:
- Tema: {rpg_tn}
- Jogador: {gs_data_param.get('player_name')} (Classe: {gs_data_param.get('player_class')}), Gênero: {player_gender_str}.
- Focos Narrativos do Jogador: {narrative_focuses_str} (Crucial! Considere estes focos ao descrever as ações e seus resultados, mostrando como o personagem se destaca).
- NPCs Conhecidos: {', '.join([f'{n} ({d})' for n, d in KNOWN_NPCS.items()]) if KNOWN_NPCS else 'Nenhum'}
- Turno Atual: {ct}/{mt} (Duração: {cdn})
- Objetivo Principal: {OBJETIVO_ATUAL}
- Inventário do Jogador: {', '.join(INVENTARIO) if INVENTARIO else 'Vazio'}
- Situação Atual (Última narração do mestre): {SITUACAO_ATUAL}

HISTÓRICO RECENTE (levando à ação atual do jogador):
{hist_f}

AÇÃO DO JOGADOR: "{user_action_param}"

SUA TAREFA (seguindo Estilo, Focos do Jogador, Gênero e Guia de Turno):
1.  NARRAÇÃO: Narre vividamente o resultado da ação do jogador (1-3 parágrafos). Se o jogador usar um Foco Narrativo implicitamente ou explicitamente, MOSTRE como isso influencia o resultado.
2.  ATUALIZAÇÕES (se houver, coloque AO FINAL da narração, cada um em NOVA LINHA, usando os prefixos EXATOS):
    NOVO_OBJETIVO: [se o objetivo principal mudar, descreva o novo objetivo conciso]
    NOVA_LOCALIZAÇÃO: [se a localização mudar significativamente, descreva a nova localização concisa]
    NOVO_INVENTÁRIO: [liste TODOS os itens ATUAIS do jogador, separados por vírgula. Ex: Espada Curta, Poção de Cura, Mapa Antigo. Se vazio, escreva: Vazio]
    NOVO_NPC: [Nome do NPC (Descrição curta da raça/profissão/aparência)] (Se um novo NPC importante for encontrado)
3.  OPÇÕES DE AÇÃO (CRUCIAL): Se não for o último turno (conforme indicado em "{turn_guidance}"), ofereça OBRIGATORIAMENTE 2-4 opções de ação CLARAS e CONCISAS para o jogador, cada uma prefixada com "OPCAO_ACAO: ". Ex: OPCAO_ACAO: Explorar a caverna escura.
4.  RESTRIÇÕES:
    -   NÃO inclua os prefixos (NOVO_OBJETIVO, OPCAO_ACAO, etc.) na narração principal. Eles devem vir APÓS a narração.
    -   NÃO mencione o número do turno (Ex: 'Turno X/Y') na sua narração principal. Apenas narre a história.
    -   Seja criativo e mantenha o engajamento do jogador!
"""
        resp_m_c = call_gemini_api(prompt_m_r, st.session_state.gemini_model_instance)
        narr_tts = resp_m_c; fin_bot_resp_ui = resp_m_c
        action_options_from_llm = []

        if "Erro:" not in resp_m_c and resp_m_c:
            lines = resp_m_c.split('\n'); fin_narr_lns = []; parsed_upds_texts = []; temp_gs_updates = {}
            original_inventory_set = set(gs_data_param.get("player_inventory",[]))
            new_npcs_found = {}

            for l_idx, l_content in enumerate(lines):
                s_line = l_content.strip()
                if s_line.startswith("NOVO_OBJETIVO:"):
                    obj = s_line.split(":", 1)[1].strip()
                    if obj and obj != gs_data_param.get("current_objective"):
                        temp_gs_updates["current_objective"] = obj
                        parsed_upds_texts.append({"type": "objetivo", "text": f"🎯 Objetivo atualizado: {obj}", "icon": "🎯", "sfx": SFX_PATHS.get("objective_updated")})
                elif s_line.startswith("NOVA_LOCALIZAÇÃO:"):
                    loc = s_line.split(":", 1)[1].strip()
                    if loc and loc != gs_data_param.get("current_location"):
                        temp_gs_updates["current_location"] = loc
                        parsed_upds_texts.append({"type": "localizacao", "text": f"📍 Nova localização: {loc}", "icon": "📍"})
                elif s_line.startswith("NOVO_INVENTÁRIO:"):
                    itms_f = s_line.split(":", 1)[1].strip()
                    itms_p = []
                    if itms_f.lower() != "vazio" and itms_f != "[]":
                        itms_s = itms_f.replace('[', '').replace(']', '')
                        itms_p = sorted([item.strip() for item in itms_s.split(',') if item.strip()])

                    current_inv_sorted = sorted(gs_data_param.get("player_inventory", []))
                    if itms_p != current_inv_sorted:
                        temp_gs_updates["player_inventory"] = itms_p
                        inv_text = ', '.join(itms_p) if itms_p else "Vazio"
                        sfx_item_to_play = SFX_PATHS.get("new_item") if set(itms_p) - original_inventory_set else None
                        parsed_upds_texts.append({"type": "inventario", "text": f"🎒 Inventário atualizado: {inv_text}", "icon": "🎒", "sfx": sfx_item_to_play})
                elif s_line.startswith("NOVO_NPC:"):
                    npc_full_str = s_line.split(":", 1)[1].strip()
                    npc_match = re.match(r"(.+?)\s*\((.*?)\)", npc_full_str)
                    if npc_match:
                        npc_name, npc_desc = npc_match.group(1).strip(), npc_match.group(2).strip()
                        if npc_name and npc_name not in gs_data_param.get("known_npcs", {}):
                            new_npcs_found[npc_name] = npc_desc
                            parsed_upds_texts.append({"type": "npc", "text": f"🗣️ Novo Contato: {npc_name} ({npc_desc})", "icon": "🗣️"})
                    elif npc_full_str and npc_full_str not in gs_data_param.get("known_npcs", {}):
                        new_npcs_found[npc_full_str] = "Descrição a ser descoberta"
                        parsed_upds_texts.append({"type": "npc", "text": f"🗣️ Novo Contato: {npc_full_str}", "icon": "🗣️"})
                elif s_line.startswith("OPCAO_ACAO:"):
                    action_option_text = s_line.split(":", 1)[1].strip()
                    if action_option_text: action_options_from_llm.append(action_option_text)
                else:
                    fin_narr_lns.append(l_content)

            narr_tts = "\n".join(fin_narr_lns).strip()
            turn_info_for_chat = f"\n\n*<span style='font-size: smaller; color: #777777;'>Turno {ct}/{mt}. {cdn}. Mestre: {gm_pers_name}.</span>*"
            fin_bot_resp_ui = narr_tts

            if parsed_upds_texts:
                ui_update_messages = []
                for upd_info in parsed_upds_texts:
                    st.toast(upd_info["text"], icon=upd_info["icon"])
                    if upd_info.get("sfx") and os.path.exists(upd_info["sfx"]): st.session_state.sfx_to_play = upd_info["sfx"]

                    if upd_info["type"] == "objetivo": ui_update_messages.append(f"<i>({upd_info['icon']} Objetivo: {temp_gs_updates['current_objective']})</i>")
                    elif upd_info["type"] == "localizacao": ui_update_messages.append(f"<i>({upd_info['icon']} Local: {temp_gs_updates['current_location']})</i>")
                    elif upd_info["type"] == "inventario":
                        inv_val_str = ', '.join(temp_gs_updates['player_inventory']) if temp_gs_updates['player_inventory'] else "Vazio"
                        ui_update_messages.append(f"<i>({upd_info['icon']} Inventário: {inv_val_str})</i>")
                    elif upd_info["type"] == "npc":

                        last_npc_name = list(new_npcs_found.keys())[-1]
                        last_npc_desc = new_npcs_found[last_npc_name]
                        ui_update_messages.append(f"<i>({upd_info['icon']} Contato: {last_npc_name} - {last_npc_desc})</i>")
                if ui_update_messages: fin_bot_resp_ui += "\n\n" + "\n".join(ui_update_messages)

            fin_bot_resp_ui += turn_info_for_chat
            st.session_state.gs.update(temp_gs_updates)
            if new_npcs_found:
                if "known_npcs" not in st.session_state.gs: st.session_state.gs["known_npcs"] = {}
                st.session_state.gs["known_npcs"].update(new_npcs_found)

            if action_options_from_llm and ct < mt :
                st.session_state.gs["current_action_options"] = action_options_from_llm
                st.session_state.gs["action_buttons_shown"] = True
            else:
                st.session_state.gs["current_action_options"] = []
                st.session_state.gs["action_buttons_shown"] = False

        st.session_state.gs["current_situation"] = narr_tts
        st.session_state.chat_history.append({"role": "assistant", "content": fin_bot_resp_ui})
        st.session_state.internal_ai_history.append({'role': 'model', 'parts': [narr_tts]})

        audio_d = await generate_audio_bytes(narr_tts)
        if audio_d: st.session_state.last_audio_bytes = audio_d

        if st.session_state.gs.get("enable_image_search", False) and narr_tts:
            print("DEBUG: Habilitando flag para processamento de imagem (IA e Google link).")
            st.session_state.gs["image_processing_in_progress"] = True

        is_button_action = user_action_param.startswith("Eu analiso o item") or user_action_param.startswith("Eu uso o item")

        if not is_button_action and not is_reroll:
            if ct == mt -2 and not gs_data_param.get("adventure_extended", False) and mt > 2:
                 st.session_state.show_extend_adventure_options = True
            elif ct >= mt :
                st.session_state.app_stage = "game_over"
            else:
                st.session_state.gs["current_turn"] += 1

    st.rerun()


if st.session_state.action_from_button:
    action_to_process = st.session_state.action_from_button
    st.session_state.action_from_button = None
    current_gs = st.session_state.get('gs', {})
    asyncio.run(process_player_action_async(action_to_process, current_gs))


with st.sidebar:
    st.header("⚙️ Configurações da Aventura")
    st.subheader("🤖 Modelo de IA (Texto)")
    current_model_text_key_sb = st.session_state.get('gemini_model_text_key', DEFAULT_GEMINI_MODEL_TEXT_KEY)
    model_keys_list = list(AVAILABLE_GEMINI_MODELS_TEXT.keys())
    default_model_idx = model_keys_list.index(current_model_text_key_sb) if current_model_text_key_sb in model_keys_list else 0

    selected_model_text_key_sb = st.radio(
        "Escolha o modelo para narração:",
        options=model_keys_list,
        format_func=lambda key: AVAILABLE_GEMINI_MODELS_TEXT[key],
        key="gemini_model_text_selector_sb",
        index=default_model_idx
    )
    if selected_model_text_key_sb != current_model_text_key_sb:
        st.session_state.gemini_model_text_key = selected_model_text_key_sb
        st.session_state.api_configured = False
        st.rerun()

    st.subheader("🔑 Chave API Gemini")
    if not st.session_state.api_configured:
        api_key_input_sb = st.text_input("Sua Chave API do Google AI Studio:", type="password", key="api_key_input_sb_main")
        if st.button("🔑 Configurar API", key="cfg_api_text_btn", type="primary", use_container_width=True):
            if api_key_input_sb:
                st.session_state.api_key = api_key_input_sb
                try:
                    genai.configure(api_key=st.session_state.api_key)
                    model_instance_sb = genai.GenerativeModel(st.session_state.gemini_model_text_key)
                    st.session_state.gemini_model_instance = model_instance_sb
                    st.session_state.api_configured = True
                    st.session_state.app_stage = "adventure_setup"
                    st.success(f"API Configurada com {AVAILABLE_GEMINI_MODELS_TEXT[st.session_state.gemini_model_text_key]}!")
                    st.rerun()
                except Exception as e:
                    st.error(f"Erro ao configurar API: {e}")
                    st.session_state.api_configured = False
            else:
                st.error("Por favor, insira uma Chave API.")
    else:
        st.success(f"API Configurada ({AVAILABLE_GEMINI_MODELS_TEXT[st.session_state.gemini_model_text_key]})")
        if st.button("🔄 Alterar Configs de API", use_container_width=True, key="change_api_config_btn"):
            current_api_key_val = st.session_state.api_key
            current_model_key_val = st.session_state.gemini_model_text_key
            init_session_state()
            st.session_state.api_key = current_api_key_val
            st.session_state.gemini_model_text_key = current_model_key_val
            st.session_state.api_configured = False
            st.session_state.app_stage = "api_config"
            st.rerun()

    st.subheader("🖼️ Imagens da Cena (Opcional)")
    current_enable_image_search = st.session_state.gs.get("enable_image_search", False)
    enable_image_search_sb = st.checkbox("Habilitar Imagem da Cena (Gerada por IA)",
                                         value=current_enable_image_search,
                                         key="image_search_toggle_sb_config",
                                         help="Gera uma imagem por IA para a cena e um link de busca no Google Imagens.")
    if enable_image_search_sb != current_enable_image_search:
        st.session_state.gs["enable_image_search"] = enable_image_search_sb
        st.rerun()

    st.markdown("---")

    if st.session_state.api_configured and st.session_state.app_stage in ["adventure_setup", "game_on", "game_over"]:
        st.subheader("✨ Crie Sua Aventura")
        gs_val = st.session_state.gs

        p_name_sb = st.text_input("Nome do Aventureiro(a):", gs_val.get("player_name", "Valente Explorador(a)"), key="player_name_sb_adv_setup")

        gender_keys_sb = list(PLAYER_GENDERS.keys()); gender_names_sb = [PLAYER_GENDERS[k] for k in gender_keys_sb]
        default_gender_key_sb = gs_val.get("player_gender_key", "neutro")
        default_gender_idx_sb = gender_keys_sb.index(default_gender_key_sb) if default_gender_key_sb in gender_keys_sb else 0
        selected_gender_name_sb = st.selectbox("Gênero do Personagem:", options=gender_names_sb, index=default_gender_idx_sb, key="player_gender_sb_adv_setup")
        player_gender_key_selected_sb = gender_keys_sb[gender_names_sb.index(selected_gender_name_sb)]

        gm_pers_keys_sb = list(GM_PERSONALITIES.keys()); gm_pers_names_sb = [GM_PERSONALITIES[k]["name"] for k in gm_pers_keys_sb]
        default_gm_pers_key_sb = gs_val.get("gm_personality_key", "neutro")
        default_gm_pers_idx_sb = gm_pers_keys_sb.index(default_gm_pers_key_sb) if default_gm_pers_key_sb in gm_pers_keys_sb else 0
        selected_gm_pers_name_sb = st.selectbox("Estilo do Mestre (IA):", options=gm_pers_names_sb, index=default_gm_pers_idx_sb, key="gm_style_sb_adv_setup")
        gm_pers_key_selected_sb = gm_pers_keys_sb[gm_pers_names_sb.index(selected_gm_pers_name_sb)]

        focus_options_sb = {key: desc for key, desc in NARRATIVE_FOCUSES.items()}
        selected_focus_keys_sb = st.multiselect("Escolha até 2 Focos Narrativos:", options=list(focus_options_sb.keys()),
                                                format_func=lambda key: focus_options_sb[key],
                                                default=gs_val.get("narrative_focus_keys", []), key="focus_multiselect_sb_adv_setup", max_selections=2)

        th_opts_keys_sb = list(RPG_THEMES_CONFIG.keys()); th_opts_lbls_sb = [RPG_THEMES_CONFIG[k] for k in th_opts_keys_sb]
        default_th_key_sb = gs_val.get("rpg_theme_key", "medieval")
        default_th_idx_sb = th_opts_keys_sb.index(default_th_key_sb) if default_th_key_sb in th_opts_keys_sb else 0
        sel_th_lbl_sb = st.selectbox("Escolha o Tema:", options=th_opts_lbls_sb, index=default_th_idx_sb, key="theme_select_sb_adv_setup")
        rpg_th_key_sb = th_opts_keys_sb[th_opts_lbls_sb.index(sel_th_lbl_sb)]

        custom_theme_name_sb = ""
        if rpg_th_key_sb == "personalizado":
            custom_theme_name_sb = st.text_input("Digite o nome do seu Tema Personalizado:",
                                                 gs_val.get("custom_theme_name", "Meu Mundo Fantástico"),
                                                 key="custom_theme_name_sb_adv_setup",
                                                 help="Ex: Vampiros na Era Vitoriana, Exploradores de Marte")

        player_gender_suffix_sb = ""
        if player_gender_key_selected_sb == "masculino": player_gender_suffix_sb = "o"
        elif player_gender_key_selected_sb == "feminino": player_gender_suffix_sb = "a"

        p_cls_sb = "N/A"
        if rpg_th_key_sb != "personalizado":
            cls_opts_sb = THEME_CLASSES.get(rpg_th_key_sb, [])
            def_cls_idx_sb = 0
            if cls_opts_sb and gs_val.get("player_class") in cls_opts_sb: def_cls_idx_sb = cls_opts_sb.index(gs_val.get("player_class"))

            p_cls_base_sb = st.radio("Escolha sua Classe:", options=cls_opts_sb or ["N/A (Defina se tema personalizado)"],
                                     index=def_cls_idx_sb, key="class_radio_sb_adv_setup")
            p_cls_sb = p_cls_base_sb
            if "(a)" in p_cls_base_sb and player_gender_suffix_sb:
                p_cls_sb = p_cls_base_sb.replace("(a)", player_gender_suffix_sb)

        custom_class_label = "Defina sua Classe:" if rpg_th_key_sb == "personalizado" else "Ou defina uma Classe Personalizada:"
        c_cls_sb = st.text_input(custom_class_label, gs_val.get("custom_class",""), key="custom_class_input_sb_adv_setup",
                                 help="Obrigatório para Tema Personalizado. Opcional para outros temas.")

        dur_opts_map_sb = {k:v["name"] for k,v in DURATION_OPTIONS_CONFIG.items()}
        def_dur_idx_sb = list(dur_opts_map_sb.keys()).index(gs_val.get("duration_key", "1")) if gs_val.get("duration_key", "1") in dur_opts_map_sb else 0
        dur_key_sb = st.radio("Duração da Aventura:", options=list(dur_opts_map_sb.keys()),
                              format_func=lambda k: dur_opts_map_sb[k], index=def_dur_idx_sb,
                              key="duration_radio_sb_adv_setup")

        enable_unexpected_events_sb_val = st.checkbox("🎲 Habilitar Eventos Inesperados",
                                                      value=gs_val.get("enable_unexpected_events", True),
                                                      key="unexpected_events_toggle_sb_val_adv_setup",
                                                      help="Se marcado, o Mestre pode introduzir surpresas ocasionais.")

        if st.button("🚀 Iniciar Nova Aventura!", type="primary", key="start_btn_adv_setup", use_container_width=True):
            final_player_class_sb = c_cls_sb.strip() if (rpg_th_key_sb == "personalizado" or c_cls_sb.strip()) else p_cls_sb
            final_rpg_theme_name = custom_theme_name_sb.strip() if rpg_th_key_sb == "personalizado" else RPG_THEMES_CONFIG[rpg_th_key_sb]

            if not p_name_sb.strip(): st.error("Insira um nome para o aventureiro(a).")
            elif rpg_th_key_sb == "personalizado" and not custom_theme_name_sb.strip(): st.error("Insira um nome para o seu Tema Personalizado.")
            elif not final_player_class_sb or final_player_class_sb == "N/A": st.error("Defina uma classe para o personagem.")
            else:
                st.session_state.gs = {
                    "player_name": p_name_sb.strip(), "player_class": final_player_class_sb,
                    "player_gender_key": player_gender_key_selected_sb,
                    "rpg_theme_key": rpg_th_key_sb,
                    "rpg_theme_config_name": final_rpg_theme_name,
                    "custom_theme_name": custom_theme_name_sb.strip() if rpg_th_key_sb == "personalizado" else "",
                    "chosen_duration_id": DURATION_OPTIONS_CONFIG[dur_key_sb]['id'],
                    "chosen_duration_name": DURATION_OPTIONS_CONFIG[dur_key_sb]['name'],
                    "max_turns": DURATION_OPTIONS_CONFIG[dur_key_sb]['turns'],
                    "extension_turns": DURATION_OPTIONS_CONFIG[dur_key_sb]['extension'],
                    "current_turn": 0, "custom_class": c_cls_sb.strip(), "duration_key": dur_key_sb,
                    "world_lore": "", "character_lore": "", "current_location": "", "current_objective": "",
                    "player_inventory": [], "known_npcs": {}, "current_situation": "",
                    "gm_personality_key": gm_pers_key_selected_sb,
                    "gm_personality_name": GM_PERSONALITIES[gm_pers_key_selected_sb]["name"],
                    "gm_personality_desc": GM_PERSONALITIES[gm_pers_key_selected_sb]["desc"],
                    "narrative_focus_keys": selected_focus_keys_sb, "adventure_extended": False,
                    "enable_unexpected_events": enable_unexpected_events_sb_val,
                    "enable_image_search": st.session_state.gs.get("enable_image_search"),
                    "current_image_search_url": None,
                    "current_image_search_terms": None,
                    "current_generated_image_bytes": None,
                    "image_processing_in_progress": False,
                    "current_action_options": [], "action_buttons_shown": False
                }
                st.session_state.bot_avatar = THEME_AVATARS.get(rpg_th_key_sb, THEME_AVATARS["default"])
                st.session_state.chat_history = [] ; st.session_state.internal_ai_history = []
                st.session_state.last_audio_bytes = None; st.session_state.sfx_to_play = None
                st.session_state.show_extend_adventure_options = False; st.session_state.adventure_extended = False
                st.session_state.last_user_action_for_reroll = None

                with st.spinner("O Mestre tece o fio do seu destino inicial..."):
                    gs_data_sb = st.session_state.gs
                    player_gender_str_val = PLAYER_GENDERS.get(gs_data_sb.get('player_gender_key', 'neutro'))
                    narrative_focuses_str_val = ", ".join([NARRATIVE_FOCUSES[f] for f in gs_data_sb.get('narrative_focus_keys',[]) if f in NARRATIVE_FOCUSES]) or "Nenhum"

                    prompt_s_val = f"""Você é um mestre de RPG experiente e criativo.
Estilo do Mestre: {gs_data_sb['gm_personality_name']} ({gs_data_sb['gm_personality_desc']}).
Tema da Aventura: {gs_data_sb['rpg_theme_config_name']}.
Jogador: {gs_data_sb['player_name']} (Classe: {gs_data_sb['player_class']}), Gênero: {player_gender_str_val}.
Focos Narrativos: {narrative_focuses_str_val}. (Use pronomes adequados ao gênero e incorpore os focos na história).
Duração: {gs_data_sb['chosen_duration_name']} (~{gs_data_sb['max_turns']} turnos).

Sua tarefa é gerar os elementos iniciais para esta aventura. Seja IMAGINATIVO, DETALHADO e CONSISTENTE com todos os parâmetros fornecidos.
Responda APENAS com os prefixos abaixo, cada um em uma nova linha, seguido pelo conteúdo:
LORE_MUNDO: [Descrição concisa (2-3 frases) do mundo da aventura, alinhado com o Tema e Estilo do Mestre.]
LORE_PERSONAGEM: [História de fundo (2-3 frases) para {gs_data_sb['player_name']}, incorporando sua Classe, Gênero e Focos Narrativos de forma orgânica.]
LOCALIZAÇÃO: [Local inicial específico e sensorial onde a aventura começa, alinhado com o Tema e Estilo.]
OBJETIVO: [Objetivo inicial claro, intrigante e alcançável (ou que inicie uma jornada maior) para o jogador.]
INVENTÁRIO: [Liste de 2 a 4 itens iniciais relevantes para o personagem e a aventura. Ex: Espada Velha, Poção de Cura Menor, Mapa Desenhado à Mão]
"""
                    resp_s_val = call_gemini_api(prompt_s_val, st.session_state.gemini_model_instance)

                    if "Erro:" not in resp_s_val and resp_s_val:
                        parsed_d_val = parse_gemini_setup_response(resp_s_val)
                        st.session_state.gs.update(parsed_d_val)
                        st.session_state.gs["current_turn"] = 1
                        gs_data_sb = st.session_state.gs

                        initial_context_for_llm_val = f"""Contexto Inicial da Aventura:
- Estilo do Mestre: {gs_data_sb['gm_personality_name']} ({gs_data_sb['gm_personality_desc']})
- Tema: {gs_data_sb['rpg_theme_config_name']}
- Mundo: {gs_data_sb.get('world_lore')}
- Personagem: {gs_data_sb['player_name']} (Classe: {gs_data_sb['player_class']}), Gênero: {PLAYER_GENDERS.get(gs_data_sb.get('player_gender_key'))}, Focos: {", ".join([NARRATIVE_FOCUSES[f] for f in gs_data_sb.get('narrative_focus_keys',[]) if f in NARRATIVE_FOCUSES]) or "Nenhum"}
- Objetivo Inicial: {gs_data_sb.get('current_objective')}
- Localização Inicial: {gs_data_sb.get('current_location')}
- Inventário Inicial: {', '.join(gs_data_sb.get('player_inventory',[]))}
- NPCs Conhecidos Inicialmente: Nenhum"""
                        st.session_state.internal_ai_history.append({'role': 'system', 'parts': [initial_context_for_llm_val]})

                        prompt_c_val = f"""Você é o Mestre de um RPG Textual. Adote o Estilo de Mestre: {gs_data_sb['gm_personality_name']} ({gs_data_sb['gm_personality_desc']}).
Esta é a PRIMEIRA narração da aventura. Deve ser MAIS LONGA (3-4 parágrafos) e DETALHADA para imergir o jogador.

Informações Base da Aventura:
- Tema: {gs_data_sb['rpg_theme_config_name']}
- Jogador: {gs_data_sb['player_name']} (Classe: {gs_data_sb['player_class']}), Gênero: {PLAYER_GENDERS.get(gs_data_sb.get('player_gender_key'))}.
- Focos Narrativos: {", ".join([NARRATIVE_FOCUSES[f] for f in gs_data_sb.get('narrative_focus_keys',[]) if f in NARRATIVE_FOCUSES]) or "Nenhum"}.
- Lore do Mundo: {gs_data_sb.get('world_lore')}
- Lore do Personagem: {gs_data_sb.get('character_lore')} (Incorpore o gênero e os focos narrativos do personagem de forma sutil na descrição).
- Objetivo Inicial: {gs_data_sb.get('current_objective')}
- Localização Inicial: {gs_data_sb.get('current_location')}
- Inventário Inicial: {', '.join(gs_data_sb.get('player_inventory',[]))}

Instruções para a Primeira Narração (seguindo Estilo, Focos e Gênero):
1.  Descreva o mundo e a situação inicial, tecendo o lore do personagem (incluindo gênero e focos relevantes de forma natural).
2.  Descreva vividamente a cena inicial (Localização), usando os sentidos (visão, audição, olfato).
3.  Introduza sutilmente o Objetivo Inicial na narrativa.
4.  Se apropriado para a cena inicial, introduza um NPC inicial com nome e breve descrição usando a tag NOVO_NPC: [Nome (Descrição)] AO FINAL da narração.
5.  **CRUCIAL (Opções de Ação):** Termine sua narração oferecendo OBRIGATORIAMENTE 2-4 opções de ação CLARAS e CONCISAS para o jogador, cada uma prefixada com "OPCAO_ACAO: " AO FINAL da narração.
6.  Use pelo menos 3-4 parágrafos para esta introdução.
7.  **Importante:** Não mencione o número do turno. Apenas narre a história.
"""
                        desc_i_val = call_gemini_api(prompt_c_val, st.session_state.gemini_model_instance)

                        if "Erro:" not in desc_i_val and desc_i_val:
                            lines_intro = desc_i_val.split('\n'); intro_narr_lines = []
                            intro_new_npcs = {}; action_options_intro = []

                            for line_intro in lines_intro:
                                s_line_intro = line_intro.strip()
                                if s_line_intro.startswith("NOVO_NPC:"):
                                    npc_full_str_intro = s_line_intro.split(":", 1)[1].strip()
                                    npc_match_intro = re.match(r"(.+?)\s*\((.*?)\)", npc_full_str_intro)
                                    if npc_match_intro:
                                        npc_name_intro, npc_desc_intro = npc_match_intro.group(1).strip(), npc_match_intro.group(2).strip()
                                        if npc_name_intro: intro_new_npcs[npc_name_intro] = npc_desc_intro
                                    elif npc_full_str_intro : intro_new_npcs[npc_full_str_intro] = "Descrição a ser descoberta"
                                elif s_line_intro.startswith("OPCAO_ACAO:"):
                                    action_option_text_intro = s_line_intro.split(":", 1)[1].strip()
                                    if action_option_text_intro: action_options_intro.append(action_option_text_intro)
                                else:
                                    intro_narr_lines.append(line_intro)

                            final_intro_narr = "\n".join(intro_narr_lines).strip()
                            ui_updates_intro_msgs = []

                            if intro_new_npcs:
                                if "known_npcs" not in st.session_state.gs: st.session_state.gs["known_npcs"] = {}
                                st.session_state.gs["known_npcs"].update(intro_new_npcs)
                                for name, desc in intro_new_npcs.items():
                                    st.toast(f"🗣️ Novo Contato: {name} ({desc})", icon="🗣️")
                                    ui_updates_intro_msgs.append(f"<i>(🗣️ Contato: {name} - {desc})</i>")

                            final_intro_ui = final_intro_narr
                            if ui_updates_intro_msgs: final_intro_ui += "\n\n" + "\n".join(ui_updates_intro_msgs)

                            turn_info_chat_intro = f"\n\n*<span style='font-size: smaller; color: #777777;'>Turno {st.session_state.gs['current_turn']}/{st.session_state.gs['max_turns']}. {st.session_state.gs['chosen_duration_name']}. Mestre: {st.session_state.gs['gm_personality_name']}.</span>*"
                            final_intro_ui += turn_info_chat_intro

                            st.session_state.gs["current_situation"] = final_intro_narr
                            st.session_state.chat_history.append({"role": "assistant", "content": final_intro_ui})
                            st.session_state.internal_ai_history.append({'role': 'model', 'parts': [final_intro_narr]})

                            if action_options_intro:
                                st.session_state.gs["current_action_options"] = action_options_intro
                                st.session_state.gs["action_buttons_shown"] = True

                            audio_d_val = asyncio.run(generate_audio_bytes(final_intro_narr))
                            if audio_d_val: st.session_state.last_audio_bytes = audio_d_val

                            if st.session_state.gs.get("enable_image_search", False) and final_intro_narr:
                                print("DEBUG: Habilitando flag para processamento de imagem (setup inicial).")
                                st.session_state.gs["image_processing_in_progress"] = True
                                st.session_state.gs["current_generated_image_bytes"] = None

                            st.session_state.app_stage = "game_on"
                            st.rerun()
                        else: st.error(f"Falha ao gerar a introdução da aventura: {desc_i_val}")
                    else: st.error(f"Falha ao configurar detalhes iniciais da aventura: {resp_s_val}")

if st.session_state.app_stage == "game_on":
    gs_game = st.session_state.gs

    # Handle Image Processing (Terms, Google Link, and AI Image Generation)
    if gs_game.get("enable_image_search", False) and \
       gs_game.get("image_processing_in_progress", False) and \
       gs_game.get("current_situation"):

        async def run_image_processing_pipeline():
            print("DEBUG: Iniciando run_image_processing_pipeline.")

            # 1. Get search terms and Google URL
            google_url, search_terms = await async_get_image_search_terms_and_google_url(
                gs_game.get("current_situation",""),
                st.session_state.gemini_model_instance
            )
            st.session_state.gs["current_image_search_url"] = google_url
            st.session_state.gs["current_image_search_terms"] = search_terms

            # 2. Generate AI Image using search terms (or narrative if terms fail)
            prompt_for_ai_image = search_terms # Prefer specific terms for AI image
            if not prompt_for_ai_image: # Fallback to narrative if terms couldn't be generated
                prompt_for_ai_image = gs_game.get("current_situation","")[:250] # Limit length

            if st.session_state.api_key and prompt_for_ai_image:
                # Use the user-specified model for image generation
                generated_bytes = await generate_ai_image_async(prompt_for_ai_image, st.session_state.api_key)
                st.session_state.gs["current_generated_image_bytes"] = generated_bytes
                print(f"DEBUG: Resultado da geração de imagem por IA: {'Bytes recebidos' if generated_bytes else 'Nenhum byte recebido'}")
            else:
                st.session_state.gs["current_generated_image_bytes"] = None
                if not st.session_state.api_key: print("DEBUG: API Key ausente para geração de imagem por IA.")
                if not prompt_for_ai_image: print("DEBUG: Prompt vazio para geração de imagem por IA.")

            st.session_state.gs["image_processing_in_progress"] = False
            print(f"DEBUG: Processamento de imagem (IA e Google Link) concluído. Rerunning.")
            st.rerun()

        with st.spinner("🖼️ Gerando imagem da cena e link para Google..."):
            asyncio.run(run_image_processing_pipeline())

    def quit_adventure_button(button_key_suffix=""):
        if st.button("🚪 Sair da Aventura", key=f"quit_adv_btn_{gs_game.get('current_turn',0)}_{button_key_suffix}", use_container_width=True):
            msg_s_quit = f"A jornada de {gs_game.get('player_name', 'Aventureiro(a)')} foi concluída por agora."
            st.session_state.chat_history.append({"role": "assistant", "content": msg_s_quit})
            st.session_state.app_stage = "game_over"
            aud_s_quit = asyncio.run(generate_audio_bytes(msg_s_quit))
            if aud_s_quit: st.session_state.last_audio_bytes = aud_s_quit
            st.rerun()

    tab_narrativa, tab_status, tab_lore = st.tabs([
        "📜 Narrativa e Ações", "📊 Status e Opções", "📚 Lore da Aventura"
    ])

    with tab_narrativa:
        with st.expander("⚡ Painel Rápido (Objetivo e Inventário)", expanded=False):
            st.markdown(f"**🎯 Objetivo Principal:** {gs_game.get('current_objective', 'Indefinido')}")
            st.markdown("**🎒 Inventário:**")
            quick_inv = gs_game.get("player_inventory", [])
            if not quick_inv: st.caption("Vazio")
            else:
                for item_idx_quick, item_name_quick in enumerate(quick_inv):
                    item_name_key_quick = re.sub(r'\W+', '', item_name_quick) + f"_quick_{item_idx_quick}_{gs_game.get('current_turn',0)}"
                    with st.container(border=True):
                        st.markdown(f"**{item_name_quick}**")
                        q_col1, q_col2 = st.columns(2)
                        with q_col1:
                            if st.button("🔍 Analisar", key=f"q_analise_{item_name_key_quick}", use_container_width=True):
                                st.session_state.action_from_button = f"Eu analiso o item: {item_name_quick}"
                                st.rerun()
                        with q_col2:
                            if st.button("🖐️ Usar", key=f"q_use_{item_name_key_quick}", use_container_width=True):
                                st.session_state.action_from_button = f"Eu uso o item: {item_name_quick}"
                                st.rerun()
                    if item_idx_quick < len(quick_inv) -1: st.markdown("<div style='margin-top: 5px;'></div>", unsafe_allow_html=True)
        st.markdown("---")

        # Display AI Generated Image
        if gs_game.get("enable_image_search", False):
            if gs_game.get("image_processing_in_progress", False):
                st.markdown("<div style='text-align: center; padding: 20px;'>🎨 Gerando imagem da cena com IA... Aguarde.</div>", unsafe_allow_html=True)
            elif gs_game.get("current_generated_image_bytes"):
                st.image(gs_game.get("current_generated_image_bytes"), caption="Imagem da Cena (Gerada por IA)", use_container_width=True)
            elif not gs_game.get("image_processing_in_progress") and gs_game.get("current_situation"):
                 st.caption("🖼️ Imagem da cena (IA) não pôde ser gerada ou não está disponível.")

        # Display Link to Google Images
        if gs_game.get("enable_image_search", False) and not gs_game.get("image_processing_in_progress"):
            if gs_game.get("current_image_search_url") and gs_game.get("current_image_search_terms"):
                search_terms_display = gs_game.get("current_image_search_terms")
                st.markdown(f"<div style='text-align: center; margin-top: 10px; margin-bottom: 10px;'><i>Termos de busca: \"{search_terms_display}\"</i><br><a href='{gs_game.get('current_image_search_url')}' target='_blank' style='color: #88c0d0;'>🔗 Ver Resultados no Google Imagens</a></div>", unsafe_allow_html=True)
            elif gs_game.get("current_situation"):
                st.caption("Não foi possível gerar termos de busca para o Google Imagens.")

        st.subheader("📜 Narrativa da Aventura")
        chat_cont_height = 380 if not gs_game.get("current_generated_image_bytes") else 200
        chat_cont = st.container(height=chat_cont_height, border=True)
        with chat_cont:
            for idx, msg_c in enumerate(st.session_state.chat_history):
                ava = st.session_state.bot_avatar if msg_c["role"] == "assistant" else "👤"
                with st.chat_message(name=msg_c["role"], avatar=ava):
                    st.markdown(msg_c["content"], unsafe_allow_html=True)

        if st.session_state.last_audio_bytes:
            st.audio(st.session_state.last_audio_bytes, format="audio/mp3")
            st.session_state.last_audio_bytes = None

        # Player action input area
        st.markdown("---")
        if gs_game.get("action_buttons_shown", False) and gs_game.get("current_action_options"):
            st.markdown("**Escolha uma ação ou digite a sua:**")
            num_options = len(gs_game["current_action_options"])
            num_cols = min(num_options, 3) if num_options > 0 else 1
            cols_options = st.columns(num_cols)
            for i, option_text in enumerate(gs_game["current_action_options"]):
                current_col_index = i % num_cols
                with cols_options[current_col_index]:
                    if st.button(option_text, key=f"action_option_{i}_{gs_game.get('current_turn',0)}", use_container_width=True):
                        st.session_state.action_from_button = option_text
                        st.rerun()

        chat_input_disabled = st.session_state.show_extend_adventure_options # Chat input is only disabled if choosing to extend
        acao_j_input_val_narr = st.chat_input("O que você faz?",
                                              key=f"act_in_{gs_game.get('current_turn',0)}",
                                              disabled=chat_input_disabled)
        if acao_j_input_val_narr:
            st.session_state.action_from_button = None
            asyncio.run(process_player_action_async(acao_j_input_val_narr, gs_game))

        if not st.session_state.show_extend_adventure_options :
            can_reroll = (len(st.session_state.chat_history) >= 1 and
                          st.session_state.chat_history[-1]["role"] == "assistant" and
                          st.session_state.last_user_action_for_reroll is not None)
            if can_reroll:
                if st.button("🔄 Refazer Última Narração", key="reroll_btn_narr", help="Pedir ao Mestre para tentar uma narração diferente para sua última ação."):
                    if st.session_state.last_user_action_for_reroll:
                        asyncio.run(process_player_action_async(st.session_state.last_user_action_for_reroll, gs_game, is_reroll=True))
                    else:
                        st.warning("Não há ação anterior do jogador para refazer a narração.")
            st.markdown("<br>", unsafe_allow_html=True)
            quit_adventure_button("narrativa")
        elif st.session_state.show_extend_adventure_options:
            st.info("A aventura está chegando perto do fim!")
            col_ext1, col_ext2 = st.columns(2)
            extension_turns_val_narr = gs_game.get("extension_turns", 10)
            with col_ext1:
                if st.button(f"➕ Estender Aventura (+{extension_turns_val_narr} turnos)", key="extend_adv_btn_narr", use_container_width=True):
                    st.session_state.gs["max_turns"] += extension_turns_val_narr
                    st.session_state.gs["chosen_duration_name"] += " (Estendida)"
                    st.session_state.gs["adventure_extended"] = True
                    st.session_state.show_extend_adventure_options = False
                    st.session_state.gs["current_turn"] += 1
                    st.toast(f"Aventura estendida em {extension_turns_val_narr} turnos!", icon="🎉")
                    st.rerun()
            with col_ext2:
                if st.button("➡️ Manter Duração Atual", key="keep_duration_narr", use_container_width=True):
                    st.session_state.show_extend_adventure_options = False
                    st.session_state.gs["current_turn"] += 1
                    st.toast("Duração mantida. A aventura continua!", icon="👍")
                    st.rerun()


    with tab_status:
        st.markdown(generate_status_markdown(gs_game), unsafe_allow_html=True)
        st.markdown("<hr style='margin-top: 5px; margin-bottom: 10px;'>", unsafe_allow_html=True)
        st.markdown("##### 🎒 Inventário Interativo:")
        player_inventory_val_status = gs_game.get("player_inventory", [])
        if not player_inventory_val_status: st.caption("Seu inventário está vazio.")
        else:
            for item_idx, item_name_val_status in enumerate(player_inventory_val_status):
                with st.container(border=True):
                    item_name_key_part_status = re.sub(r'\W+', '', item_name_val_status) + f"_{item_idx}_{gs_game.get('current_turn',0)}"
                    st.markdown(f"**{item_name_val_status}**")
                    col_btn_item1, col_btn_item2 = st.columns(2)
                    with col_btn_item1:
                        if st.button(f"🔍 Analisar", key=f"analise_item_status_{item_name_key_part_status}", help=f"Analisar: {item_name_val_status}", use_container_width=True):
                            st.session_state.action_from_button = f"Eu analiso o item: {item_name_val_status}"
                            st.rerun()
                    with col_btn_item2:
                        if st.button(f"🖐️ Usar", key=f"use_item_status_{item_name_key_part_status}", help=f"Tentar usar: {item_name_val_status}", use_container_width=True):
                            st.session_state.action_from_button = f"Eu uso o item: {item_name_val_status}"
                            st.rerun()
                if item_idx < len(player_inventory_val_status) - 1: st.markdown("<div style='margin-top: 10px;'></div>", unsafe_allow_html=True)
        st.markdown("<hr style='margin-top: 10px; margin-bottom: 5px;'>", unsafe_allow_html=True)
        quit_adventure_button("status")

    with tab_lore:
        st.markdown(f"<div style='font-size: 1.05em;'>", unsafe_allow_html=True)
        st.markdown(f"<h4>Estilo do Mestre: {gs_game.get('gm_personality_name', 'N/A')}</h4>", unsafe_allow_html=True)
        st.caption(f"{gs_game.get('gm_personality_desc', 'Não definido.')}")
        st.markdown("---")
        focus_texts_val_lore = [NARRATIVE_FOCUSES[key] for key in gs_game.get('narrative_focus_keys', []) if key in NARRATIVE_FOCUSES]
        focus_display_lore_val = ', '.join(focus_texts_val_lore) if focus_texts_val_lore else "Nenhum"
        st.markdown(f"<h4>Seus Focos Narrativos:</h4>", unsafe_allow_html=True)
        st.caption(focus_display_lore_val)
        st.markdown("---")
        known_npcs_lore = gs_game.get("known_npcs", {})
        if known_npcs_lore:
            st.markdown(f"<h4>Contatos e NPCs Conhecidos:</h4>", unsafe_allow_html=True)
            for npc_name, npc_desc in known_npcs_lore.items():
                st.markdown(f" - **{npc_name}:** *{npc_desc}*", unsafe_allow_html=True)
            st.markdown("---")
        st.markdown(f"<h4>Sobre Este Mundo... ({gs_game.get('rpg_theme_config_name', 'N/A')})</h4>", unsafe_allow_html=True)
        st.caption(f"{gs_game.get('world_lore', 'N/D')}")
        st.markdown("---")
        st.markdown(f"<h4>Sua História, {gs_game.get('player_name', 'Aventureiro(a)')}...</h4>", unsafe_allow_html=True)
        st.caption(f"**Gênero:** {PLAYER_GENDERS.get(gs_game.get('player_gender_key', 'neutro'), 'N/D')}, **Classe:** {gs_game.get('player_class', 'N/D')}")
        st.caption(f"{gs_game.get('character_lore', 'N/D')}")
        st.markdown("---")
        st.markdown(f"<h4>Seu Objetivo Principal</h4>", unsafe_allow_html=True)
        st.info(f"{gs_game.get('current_objective', 'N/D')}")
        st.markdown("</div>", unsafe_allow_html=True)
        st.markdown("<br>", unsafe_allow_html=True)
        quit_adventure_button("lore")

elif st.session_state.app_stage == "game_over":
    st.balloons(); st.header("🎉 Fim da Aventura! 🎉"); gs_end = st.session_state.gs
    status_html_final_val_end = generate_status_markdown(gs_end)
    inventory_final_display_list_val_end = gs_end.get('player_inventory', [])
    inventory_final_str_val_end = f"**🎒 Inventário Final:** {(', '.join(inventory_final_display_list_val_end) if inventory_final_display_list_val_end else 'Vazio')}"

    hr_index_end = status_html_final_val_end.rfind("<hr")
    status_html_final_with_inv_val_end = status_html_final_val_end
    if hr_index_end != -1:
        closing_div_after_obj_loc_end = status_html_final_val_end.find("</div>", hr_index_end)
        if closing_div_after_obj_loc_end != -1:
            status_html_final_with_inv_val_end = (
                status_html_final_val_end[:closing_div_after_obj_loc_end] +
                f"<br>{inventory_final_str_val_end}" +
                status_html_final_val_end[closing_div_after_obj_loc_end:]
            )
    st.markdown(status_html_final_with_inv_val_end, unsafe_allow_html=True)

    st.markdown("### 📜 Histórico Final da Aventura:"); chat_cont_end_val_end=st.container(height=400, border=True)
    with chat_cont_end_val_end:
        bot_ava_val_end=THEME_AVATARS.get(gs_end.get("rpg_theme_key"), THEME_AVATARS["default"])
        for idx_e,msg_e_val_end in enumerate(st.session_state.chat_history):
            with st.chat_message(msg_e_val_end["role"], avatar=(bot_ava_val_end if msg_e_val_end["role"] == "assistant" else "👤")):
                st.markdown(msg_e_val_end["content"], unsafe_allow_html=True)
    if st.session_state.last_audio_bytes:
        st.audio(st.session_state.last_audio_bytes, format="audio/mp3"); st.session_state.last_audio_bytes = None
    st.markdown("---")
    if st.button("🔄 Iniciar Nova Aventura", type="primary", use_container_width=True, key="new_adv_game_over_btn"):
        current_api_key_val_end = st.session_state.api_key
        api_cfg_status_val_end = st.session_state.api_configured
        current_model_key_val_end_game = st.session_state.gemini_model_text_key
        current_gemini_instance = st.session_state.gemini_model_instance
        current_enable_image_search_val = st.session_state.gs.get("enable_image_search", False)

        init_session_state()

        st.session_state.api_key = current_api_key_val_end
        st.session_state.api_configured = api_cfg_status_val_end
        st.session_state.gemini_model_text_key = current_model_key_val_end_game
        st.session_state.gemini_model_instance = current_gemini_instance
        st.session_state.gs["enable_image_search"] = current_enable_image_search_val

        st.session_state.app_stage = "adventure_setup" if api_cfg_status_val_end else "api_config"
        st.rerun()

elif st.session_state.app_stage == "api_config":
    if not _edge_tts_available: st.warning("Biblioteca `edge-tts` não encontrada. A funcionalidade de voz estará indisponível.", icon="🔊")
    st.info("👋 Bem-vindo! Configure sua Chave API Gemini e o Modelo de IA na barra lateral para começar sua aventura.", icon="🔑")
    st.markdown("---")
    st.subheader("🎲 Bem-vindo ao RPG Textual com IA Gemini!")
    st.markdown("""
Embarque em jornadas épicas moldadas por suas decisões e pela criatividade ilimitada da Inteligência Artificial Gemini!
Prepare-se para explorar mundos fantásticos, enfrentar desafios perigosos e interagir com personagens memoráveis.

**Como começar:**
1.  Na barra lateral à esquerda:
    * Escolha o **Modelo de IA Gemini** desejado para a narração (Flash para rapidez, Pro para robustez).
    * Insira sua **Chave API do Google AI Studio** (esta chave será usada tanto para texto quanto para a tentativa de geração de imagem com o modelo `gemini-2.0-flash-preview-image-generation`).
    * Clique em "Configurar API".
2.  Após a configuração da API:
    * Defina os detalhes da sua aventura (nome, tema, classe, etc.).
    * Opcionalmente, habilite a "Habilitar Imagem da Cena (Gerada por IA)" para uma experiência mais visual.
    * Clique em "Iniciar Nova Aventura!" e deixe a imaginação fluir!

**Dica:** Se a opção de imagem estiver habilitada, o sistema tentará gerar uma imagem da cena usando IA. Um link para mais imagens no Google também será fornecido.
    """)

In [24]:
from google.colab import userdata
from pyngrok import ngrok, conf
import os
import time

NGROK_AUTHTOKEN_FROM_SECRETS = userdata.get('NGROK_AUTHTOKEN')

if not NGROK_AUTHTOKEN_FROM_SECRETS:
    print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    print("ERRO: Token NGROK_AUTHTOKEN não encontrado nos Secrets do Colab.")
    print("Por favor, adicione seu token ngrok aos Secrets do Colab com o nome NGROK_AUTHTOKEN.")
    print("1. Clique no ícone de chave (🔑) na barra lateral esquerda do Colab.")
    print("2. Clique em '+ ADICIONAR UM NOVO SECRET'.")
    print("3. Nome: NGROK_AUTHTOKEN")
    print("4. Valor: SEU_TOKEN_NGROK_COMPLETO_AQUI (ex: 2xL7RWu7GhTHHLoFNYkjTg495KD_5mSLm4oYdNMpLYPWEDB1X)")
    print("5. Marque 'Acesso ao notebook'.")
    print("6. Salve e execute esta célula novamente.")
    print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
else:
    print(f"Token NGROK_AUTHTOKEN carregado dos Secrets do Colab.")
    conf.get_default().auth_token = NGROK_AUTHTOKEN_FROM_SECRETS

    print("Tentando encerrar túneis ngrok anteriores...")
    try:
        active_tunnels = ngrok.get_tunnels()
        if active_tunnels:
            for tunnel in active_tunnels:
                public_url = tunnel.public_url
                ngrok.disconnect(public_url)
                print(f"Túnel {public_url} desconectado.")
            ngrok.kill() # Garante que o processo ngrok seja finalizado
            print("Processo ngrok anterior finalizado.")
        else:
            print("Nenhum túnel ngrok ativo encontrado para desconectar.")
        time.sleep(2) # Pequena pausa para garantir que os túneis sejam fechados
    except Exception as e:
        print(f"Erro ao tentar desconectar/finalizar túneis ngrok anteriores: {e}")
        print("Isso pode ser normal se nenhum processo ngrok estava rodando.")

    print("Tentativa de finalizar processos Streamlit anteriores...")
    os.system("pkill -f \"streamlit run rpg_streamlit_app.py\"")
    time.sleep(3)

    print("Iniciando Streamlit app em background...")
    os.system("streamlit run rpg_streamlit_app.py --server.port 8501 --server.headless true --server.enableCORS false &")

    print("Iniciando ngrok... Aguarde alguns segundos.")
    time.sleep(10) # Aumentar um pouco o tempo para o Streamlit iniciar completamente

    try:
        public_url_tunnel = ngrok.connect(8501, proto="http")
        print("=" * 70)
        print(f"🚀 Sua aplicação RPG Streamlit está rodando em: {public_url_tunnel.public_url}")
        print("Copie e cole o link acima no seu navegador!")
        print("=" * 70)
    except Exception as e:
        print(f"Erro crítico ao iniciar o túnel ngrok: {e}")
        print("\nPossíveis Soluções:")
        print("1. Verifique se o authtoken do ngrok nos Secrets do Colab está correto e ativo.")
        print("2. O ngrok pode ter atingido um limite de conexões ou outra restrição da conta gratuita.")
        print("3. Certifique-se de que a internet do Colab está funcionando.")
        print("4. Tente executar esta célula novamente após alguns segundos.")
        print("5. Verifique o dashboard do ngrok (http://127.0.0.1:4040 - pode não ser acessível diretamente no Colab, mas o log do ngrok pode dar pistas).")

Token NGROK_AUTHTOKEN carregado dos Secrets do Colab.
Tentando encerrar túneis ngrok anteriores...
Túnel https://df6e-104-196-171-177.ngrok-free.app desconectado.
Processo ngrok anterior finalizado.
Tentativa de finalizar processos Streamlit anteriores...
Iniciando Streamlit app em background...
Iniciando ngrok... Aguarde alguns segundos.
🚀 Sua aplicação RPG Streamlit está rodando em: https://9ec6-104-196-171-177.ngrok-free.app
Copie e cole o link acima no seu navegador!
